<a href="https://colab.research.google.com/github/avichaychriqui/HeBERT/blob/main/data/sentiment_analysis_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install shap==0.36.0
# !pip install transformers==3.3.1
# !pip install datasets

In [2]:
import shap
import transformers
import torch
import numpy as np
import scipy as sp
import pandas as pd

In [3]:
from datasets import load_dataset
dataset = load_dataset('text', data_files='text_example.txt', )


Using custom data configuration default
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-dba86d70c11ab66c/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)


In [4]:
dataset['train']['text'][::-5]

['מה אוכלים היום?',
 'קפה זה טעים',
 'אני אוהב את העולם',
 '" וואו, מתאים לספר השיאים של גינס. מקסים ומרגש!!!! . לא יודעת מי המתורגמנית. אבל יודעת בוודאות כמה זה חשוב!!!! כל כך חשוב. יש מתורגמנית שכולם מכירים מהחדשות בטלויזיה, אפרת נגר המדהימה שהיא גם דולה... עבודת קודש אמיתית. מזל טוב יקרים. הרבה בריאות ושמחה."',
 ' והרכבת ? זה זמן עוד יותר טוב לרכבת שלא פעילה בכלל.. כל הכבוד זה הזמן אבל350 אלף כלי רכב בתקופת הקורונה????',
 ' והבס שלך לא עבריין??? 3 תיקי אישומים. ',
 '" וביבי יצילנו מידם... הוא ולא שרף. 1. תחילה הסתה וגזענות - איבדתה את הרוב של הכנסת2.עתה נגד הדמוקרטיה - הצלחתה לאחד מולך את כל האופוזציה, כולל מבית3. צעד נוסף באותו כיוון - אשריך !"']

In [5]:

from transformers import AutoTokenizer, AutoModel, pipeline

# how to use?
sentiment_analysis = pipeline(
    "sentiment-analysis",
    model="avichr/heBERT_sentiment_analysis",
    tokenizer="avichr/heBERT_sentiment_analysis", 
      return_all_scores = True, 
)


In [6]:
for i in dataset['train']['text'][0:5]:
  print(i, sentiment_analysis(i), sep='\n')


" וביבי יצילנו מידם... הוא ולא שרף. 1. תחילה הסתה וגזענות - איבדתה את הרוב של הכנסת2.עתה נגד הדמוקרטיה - הצלחתה לאחד מולך את כל האופוזציה, כולל מבית3. צעד נוסף באותו כיוון - אשריך !"
[[{'label': 'LABEL_0', 'score': 0.9999509453773499}, {'label': 'LABEL_1', 'score': 4.905865716864355e-05}]]
" וביבי שותק. הם לא נעמה. הם נמצאים בבידוד ולא במאסר, הם בסביבה שדואגים להם והם לא נמצאים בסכנה מיידית כלשהי, ובנוסף הם עלולים להוות סיכון לאחרים. עם כל אי הנוחות, להישאר על הספינה זה הפתרון הטוב ביותר כרגע"
[[{'label': 'LABEL_0', 'score': 0.3695709705352783}, {'label': 'LABEL_1', 'score': 0.6304290890693665}]]
 ובישראל חסר מסביר כי ביבי לא ייתן לאף אחד לגנוב לו את ההצגה. היועץ הלאומי של שישה נשיאים ברציפות. מיגור הסארס בארה'ב נזקף לזכותו. הוא הפילטר להכרזות ההזויות של הנשיא הנוכחי (רק מלהזכיר את השם שלו אני חוטף קורו......)
[[{'label': 'LABEL_0', 'score': 0.9999604225158691}, {'label': 'LABEL_1', 'score': 3.955313513870351e-05}]]
" וגם הרוב עצות יעילות וטובות, מנסיון "
[[{'label': 'LABEL_0', 'score'

In [7]:


# load a BERT sentiment analysis model
tokenizer = transformers.BertTokenizerFast.from_pretrained("avichr/heBERT_sentiment_analysis")
model = transformers.BertForSequenceClassification.from_pretrained("avichr/heBERT_sentiment_analysis").cuda()

# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, pad_to_max_length=True, max_length=500, truncation=True, padding='longest') for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

# explain the model's predictions on IMDB reviews
train_dataset = dataset['train']['text']
shap_values = explainer(train_dataset)


explainers.Partition is still in an alpha state, so use with caution...


In [8]:
shap.plots.text(shap_values)
